In [15]:
# STEP 1: Install dependencies
!pip install -q sentence-transformers

# STEP 2: Import libraries
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# STEP 3: Upload your CSV file


# STEP 4: Load the CSV
df = pd.read_csv("app_reviews_analysis.csv")

# STEP 5: Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# STEP 6: Compute semantic similarity between App Info Modal and User Review Analysis
def compute_similarity(text1, text2):
    if pd.isna(text1) or pd.isna(text2):
        return 0.0
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
    return round(similarity, 4)

similarities = []
recommendations = []

for _, row in df.iterrows():
    modal = row['App Info Modal']
    review = row['User Review Analysis']
    sim = compute_similarity(modal, review)
    similarities.append(sim)

    # Trust decision based on similarity threshold
    if sim > 0.7:
        recommendations.append("Trust Developers")
    else:
        recommendations.append("Trust Users")

df["Semantic Similarity"] = similarities
df["Trustworthiness Recommendation"] = recommendations


def filter_difference_analysis(diff_text, decision):
    if pd.isna(diff_text):
        return diff_text

    lines = diff_text.split("\n")

    def is_dev_line(line):
        return line.strip().lower().startswith("- **developer")

    def is_user_line(line):
        return line.strip().lower().startswith("- **user")

    if decision == "Trust Users":
        filtered = [line for line in lines if not is_dev_line(line)]
    elif decision == "Trust Developers":
        filtered = [line for line in lines if not is_user_line(line)]
    else:
        filtered = lines

    return "\n".join(filtered)




df["Filtered Difference Analysis"] = [
    filter_difference_analysis(row["Difference Analysis"], row["Trustworthiness Recommendation"])
    for _, row in df.iterrows()
]

# STEP 8: Export the updated dataframe
df.to_csv("app_truthfulness_analysis.csv", index=False)
print("✅ Done! You can download your final file now:")

# STEP 9: Enable file download
#files.download("app_truthfulness_analysis.csv")


✅ Done! You can download your final file now:
